<a href="https://colab.research.google.com/github/Logeshwaran32/Bizcard/blob/main/Bizcard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install streamlit_option_menu
!pip install EasyOCR
!pip install mysql
!pip install opencv-python-headless
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wh

In [56]:
%%writefile app.py
import pandas as pd
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
import sqlite3 as sql
#import mysql.connector as sql
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt
import re
from pyngrok import ngrok

# SETTING PAGE CONFIGURATIONS
icon = Image.open("/content/streamlit-logo-primary-colormark-darktext.png")
st.set_page_config(page_title="BizCardX: Extracting Business Card Data with OCR",
                   page_icon=icon,
                   layout="wide",
                   initial_sidebar_state="expanded",
                   menu_items={'About': """# This OCR app is created by *Logeshwaran M*!"""})
st.markdown("<h1 style='text-align: center; color: blue;'>BizCardX: Extracting Business Card Data with OCR</h1>",
            unsafe_allow_html=True)

# SETTING-UP BACKGROUND IMAGE
def setting_bg():
    st.markdown(f""" <style>.stApp {{
                        background:url("https://wallpapers.com/images/featured/plain-zoom-background-d3zz0xne0jlqiepg.jpg");
                        background-size: cover}}
                     </style>""", unsafe_allow_html=True)

setting_bg()

# CREATING OPTION MENU
selected = option_menu(None, ["Home", "Upload & Extract", "Modify"],
                       icons=["house", "cloud-upload", "pencil-square"],
                       default_index=0,
                       orientation="horizontal",
                       styles={"nav-link": {"font-size": "35px", "text-align": "centre", "margin": "-2px",
                                           "--hover-color": "#6495ED"},
                               "icon": {"font-size": "35px"},
                               "container": {"max-width": "6000px"},
                               "nav-link-selected": {"background-color": "#6495ED"}})

# INITIALIZING THE EasyOCR READER
reader = easyocr.Reader(['en'],gpu=False)

# CONNECTING WITH MYSQL DATABASE
#mysql_conn = sql.connect(':memory:')
#mysql_cursor = mysql_conn.cursor()
# CONNECTING WITH SQLITE DATABASE
sqlite_conn = sql.connect('mlw_database.db')
sqlite_cursor = sqlite_conn.cursor()

# TABLE CREATION
sqlite_cursor.execute('''CREATE TABLE IF NOT EXISTS card_data
                   (id INTEGER PRIMARY KEY AUTOINCREMENT,
                    company_name TEXT,
                    card_holder TEXT,
                    designation TEXT,
                    mobile_number VARCHAR(50),
                    email TEXT,
                    website TEXT,
                    area TEXT,
                    city TEXT,
                    state TEXT,
                    pin_code VARCHAR(10),
                    image LONGBLOB
                    )''')

# HOME MENU
if selected == "Home":
    col1 , col2 = st.columns(2)
    with col1:
        st.image(Image.open("/content/Business-Cards-extraction-2.png"),width=500)
        st.markdown("## :green[**Technologies Used :**] Python,easy OCR, Streamlit, SQL, Pandas")
    with col2:
       st.write("## :green[**About :**] Bizcard is a Python application designed to extract information from business cards.")
       st.write('## The main purpose of Bizcard is to automate the process of extracting key details from business card images, such as the name, designation, company, contact information, and other relevant data. By leveraging the power of OCR (Optical Character Recognition) provided by EasyOCR, Bizcard is able to extract text from the images.')

# UPLOAD AND EXTRACT MENU

if selected == "Upload & Extract":
    if st.button(":blue[Already stored data]"):
        sqlite_cursor.execute(
            "select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data")
        updated_df = pd.DataFrame(sqlite_cursor.fetchall(),
                                  columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number",
                                           "Email",
                                           "Website", "Area", "City", "State", "Pin_Code"])
        st.write(updated_df)
    st.subheader(":blue[Upload a Business Card]")
    uploaded_card = st.file_uploader("upload here", label_visibility="collapsed", type=["png", "jpeg", "jpg"])

    if uploaded_card is not None:
        def save_card(uploaded_card):
            uploaded_cards_dir = os.path.join(os.getcwd(), "uploaded_cards")

            # Create the directory if it doesn't exist
            os.makedirs(uploaded_cards_dir, exist_ok=True)

            with open(os.path.join(uploaded_cards_dir, uploaded_card.name), "wb") as f:
                f.write(uploaded_card.getbuffer())

        save_card(uploaded_card)

        def image_preview(image, res):
            for (bbox, text, prob) in res:
                # unpack the bounding box
                (tl, tr, br, bl) = bbox
                tl = (int(tl[0]), int(tl[1]))
                tr = (int(tr[0]), int(tr[1]))
                br = (int(br[0]), int(br[1]))
                bl = (int(bl[0]), int(bl[1]))
                cv2.rectangle(image, tl, br, (0, 255, 0), 2)
                cv2.putText(image, text, (tl[0], tl[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            plt.rcParams['figure.figsize'] = (15, 15)
            plt.axis('off')
            plt.imshow(image)


        # DISPLAYING THE UPLOADED CARD
        col1, col2 = st.columns(2, gap="large")
        with col1:
            st.markdown("#     ")
            st.markdown("#     ")
            st.markdown("### You have uploaded the card")
            st.image(uploaded_card)
        # DISPLAYING THE CARD WITH HIGHLIGHTS
        with col2:
            st.markdown("#     ")
            st.markdown("#     ")
            with st.spinner("Please wait processing image..."):
                st.set_option('deprecation.showPyplotGlobalUse', False)
                #saved_img = os.getcwd() + "\\" + "uploaded_cards" + "\\" + uploaded_card.name
                saved_img = os.path.join(os.getcwd(), "uploaded_cards", uploaded_card.name)
                uploaded_cards_dir = os.path.join(os.getcwd(), "uploaded_cards")
                image = cv2.imread(saved_img)
                res = reader.readtext(saved_img)
                st.markdown("### Image Processed and Data Extracted")
                st.pyplot(image_preview(image, res))

                # easy OCR
        #saved_img = os.getcwd() + "\\" + "uploaded_cards" + "\\" + uploaded_card.name
        saved_img = os.path.join(os.getcwd(), "uploaded_cards", uploaded_card.name)
        result = reader.readtext(saved_img, detail=0, paragraph=False)


        # CONVERTING IMAGE TO BINARY TO UPLOAD TO SQL DATABASE
        def img_to_binary(file):
            # Convert image data to binary format
            with open(file, 'rb') as file:
                binaryData = file.read()
            return binaryData


        data = {"company_name": [],
                "card_holder": [],
                "designation": [],
                "mobile_number": [],
                "email": [],
                "website": [],
                "area": [],
                "city": [],
                "state": [],
                "pin_code": [],
                "image": img_to_binary(saved_img)
                }


        def get_data(res):
            for ind, i in enumerate(res):

                # To get WEBSITE_URL
                if "www " in i.lower() or "www." in i.lower():
                    data["website"].append(i)
                elif "WWW" in i:
                    data["website"] = res[4] + "." + res[5]

                # To get EMAIL ID
                elif "@" in i:
                    data["email"].append(i)

                # To get MOBILE NUMBER
                elif "-" in i:
                    data["mobile_number"].append(i)
                    if len(data["mobile_number"]) == 2:
                        data["mobile_number"] = " & ".join(data["mobile_number"])

                # To get COMPANY NAME
                elif ind == len(res) - 1:
                    data["company_name"].append(i)

                # To get CARD HOLDER NAME
                elif ind == 0:
                    data["card_holder"].append(i)

                # To get DESIGNATION
                elif ind == 1:
                    data["designation"].append(i)

                # To get AREA
                if re.findall('^[0-9].+, [a-zA-Z]+', i):
                    data["area"].append(i.split(',')[0])
                elif re.findall('[0-9] [a-zA-Z]+', i):
                    data["area"].append(i)

                # To get CITY NAME
                match1 = re.findall('.+St , ([a-zA-Z]+).+', i)
                match2 = re.findall('.+St,, ([a-zA-Z]+).+', i)
                match3 = re.findall('^[E].*', i)
                if match1:
                    data["city"].append(match1[0])
                elif match2:
                    data["city"].append(match2[0])
                elif match3:
                    data["city"].append(match3[0])

                # To get STATE
                state_match = re.findall('[a-zA-Z]{9} +[0-9]', i)
                if state_match:
                    data["state"].append(i[:9])
                elif re.findall('^[0-9].+, ([a-zA-Z]+);', i):
                    data["state"].append(i.split()[-1])
                if len(data["state"]) == 2:
                    data["state"].pop(0)

                # To get PINCODE
                if len(i) >= 6 and i.isdigit():
                    data["pin_code"].append(i)
                elif re.findall('[a-zA-Z]{9} +[0-9]', i):
                    data["pin_code"].append(i[10:])


        get_data(result)


        # FUNCTION TO CREATE DATAFRAME
        def create_df(data):
            df = pd.DataFrame(data)
            return df


        df = create_df(data)
        st.success("### Data Extracted!")
        st.write(df)

        if st.button("Upload to Database"):
            for i, row in df.iterrows():
                # here %S means string values
                sql = '''INSERT INTO card_data(company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code,image)
                VALUES (?,?,?,?,?,?,?,?,?,?,?)'''
                sqlite_cursor.execute(sql, tuple(row))
                # the connection is not auto committed by default, so we must commit to save our changes
                sqlite_conn.commit()
                st.success("#### Uploaded to database successfully!")

        if st.button(":blue[View updated data]"):
            sqlite_cursor.execute("select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data")
            updated_df = pd.DataFrame(sqlite_cursor.fetchall(),
                                          columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number",
                                                   "Email",
                                                   "Website", "Area", "City", "State", "Pin_Code"])
            st.write(updated_df)

# MODIFY MENU
if selected == "Modify":
    st.subheader(':blue[You can view , alter or delete the extracted data in this app]')
    select = option_menu(None,
                         options=["ALTER", "DELETE"],
                         default_index=0,
                         orientation="horizontal",
                         styles={"container": {"width": "100%"},
                                 "nav-link": {"font-size": "20px", "text-align": "center", "margin": "-2px"},
                                 "nav-link-selected": {"background-color": "#6495ED"}})

    if select == "ALTER":
        st.markdown(":blue[Alter the data here]")
        try:
            sqlite_cursor.execute("SELECT DISTINCT card_holder FROM card_data")
            result = sqlite_cursor.fetchall()
            business_cards = [row[0] for row in result]

            selected_card = st.selectbox("**Select a card**", ["None"] + business_cards)
            if selected_card == "None":
                st.write("No card selected.")
            else:
                st.markdown("#### Update or modify any data below")
                sqlite_cursor.execute(
                    "select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data WHERE card_holder=?",
                    (selected_card,))
                result = sqlite_cursor.fetchone()

                # DISPLAYING ALL THE INFORMATIONS
                company_name = st.text_input("Company_Name", result[0])
                card_holder = st.text_input("Card_Holder", result[1])
                designation = st.text_input("Designation", result[2])
                mobile_number = st.text_input("Mobile_Number", result[3])
                email = st.text_input("Email", result[4])
                website = st.text_input("Website", result[5])
                area = st.text_input("Area", result[6])
                city = st.text_input("City", result[7])
                state = st.text_input("State", result[8])
                pin_code = st.text_input("Pin_Code", result[9])


                if st.button(":blue[Commit changes to DB]"):

                    # Update the information for the selected business card in the database
                    sqlite_cursor.execute("""UPDATE card_data SET company_name=?,card_holder=?,designation=?,mobile_number=?,email=?,website=?,area=?,city=?,state=?,pin_code=?
                                    WHERE card_holder=?""", (company_name, card_holder, designation, mobile_number, email, website, area, city, state, pin_code,
                    selected_card))
                    sqlite_conn.commit()
                    st.success("Information updated in database successfully.")

            if st.button(":blue[View updated data]"):
                sqlite_cursor.execute(
                    "select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data")
                updated_df = pd.DataFrame(sqlite_cursor.fetchall(),
                                          columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number",
                                                   "Email",
                                                   "Website", "Area", "City", "State", "Pin_Code"])
                st.write(updated_df)

        except Exception as e:
            st.warning(f"Error: {e}")

    if select == "DELETE":
        st.subheader(":blue[Delete the data]")
        try:
            sqlite_cursor.execute("SELECT card_holder FROM card_data")
            result = sqlite_cursor.fetchall()
            business_cards = {}
            for row in result:
                business_cards[row[0]] = row[0]
            options = ["None"] + list(business_cards.keys())
            selected_card = st.selectbox("**Select a card**", options)
            if selected_card == "None":
                st.write("No card selected.")
            else:
                st.write(f"### You have selected :green[**{selected_card}'s**] card to delete")
                st.write("#### Proceed to delete this card?")
                if st.button("Yes Delete Business Card"):
                    sqlite_cursor.execute(f"DELETE FROM card_data WHERE card_holder='{selected_card}'")
                    sqlite_conn.commit()
                    st.success("Business card information deleted from database.")

            if st.button(":blue[View updated data]"):
                sqlite_cursor.execute(
                    "select company_name,card_holder,designation,mobile_number,email,website,area,city,state,pin_code from card_data")
                updated_df = pd.DataFrame(sqlite_cursor.fetchall(),
                                          columns=["Company_Name", "Card_Holder", "Designation", "Mobile_Number",
                                                   "Email",
                                                   "Website", "Area", "City", "State", "Pin_Code"])
                st.write(updated_df)

        except:
            st.warning("There is no data available in the database")

#from pyngrok import ngrok

# Add this line to ensure some interaction in the Streamlit app
st.balloons()

# Set your ngrok authtoken
#ngrok.set_auth_token("2c9aDiM5RxXB3yGP7TtRxIfuzp3_6uzgxsNh4FKPmZf4vRnz")

# Set up ngrok to tunnel the Streamlit app on port 8501
#public_url = ngrok.connect(addr='8501', proto='http')

# Print the public URL
#print('Streamlit app is live at:', public_url)


Overwriting app.py


In [40]:
# Restart Streamlit app
!streamlit run app.py &>/dev/null&

In [41]:
!ngrok config add-authtoken 2bthDyyaiMiZYDL2k8MDnRD8GGs_6tghXCdi23Y49kGT8hSgc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [42]:
from pyngrok import ngrok
public_url = ngrok.connect(addr='8501')
print(public_url)

NgrokTunnel: "https://c1b1-34-147-56-23.ngrok-free.app" -> "http://localhost:8501"


In [45]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('mlw_database.db')
cursor = conn.cursor()

In [46]:
# Execute SQL query to fetch data from the card_data table
cursor.execute("SELECT * FROM card_data")

# Fetch all rows from the result set
rows = cursor.fetchall()

# Display the fetched data
for row in rows:
    print(row)

(1, 'digitals', 'Selva', 'DATA MANAGER', '+123-456-7890 & +123-456-7891', 'hello@XYZ1.com', 'WWW XYZI.com', '123 ABC St ', 'Chennai', 'TamilNadu', '600113', b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xec\x00\x00\x02N\x08\x06\x00\x00\x00-z\x8d\xd9\x00\x00\x00\tpHYs\x00\x00.#\x00\x00.#\x01x\xa5?v\x00\x00\x04\x7fiTXtXML:com.adobe.xmp\x00\x00\x00\x00\x00<?xpacket begin=\'\xef\xbb\xbf\' id=\'W5M0MpCehiHzreSzNTczkc9d\'?>\n<x:xmpmeta xmlns:x=\'adobe:ns:meta/\'>\n<rdf:RDF xmlns:rdf=\'http://www.w3.org/1999/02/22-rdf-syntax-ns#\'>\n\n <rdf:Description rdf:about=\'\'\n  xmlns:Attrib=\'http://ns.attribution.com/ads/1.0/\'>\n  <Attrib:Ads>\n   <rdf:Seq>\n    <rdf:li rdf:parseType=\'Resource\'>\n     <Attrib:Created>2023-02-27</Attrib:Created>\n     <Attrib:ExtId>9ac1f0a6-56e7-49da-912d-41894ed0c0f1</Attrib:ExtId>\n     <Attrib:FbId>525265914179580</Attrib:FbId>\n     <Attrib:TouchType>2</Attrib:TouchType>\n    </rdf:li>\n   </rdf:Seq>\n  </Attrib:Ads>\n </rdf:Description>\n\n <rdf:Descripti